In [3]:

from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.master("spark://spark:7077").appName("PySparkTest").getOrCreate()
# create a simple DataFrame
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3), ("Dave", 4)]
df = spark.createDataFrame(data, ["Name", "Age"])

# show the DataFrame
df.show()

# stop the SparkSession
spark.stop()

+-------+---+
|   Name|Age|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
|   Dave|  4|
+-------+---+

